In [ ]:
!pip install opencv-python dlib imutils


In [ ]:
import os

MODEL_URL = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"

# Download and extract the model
!wget $MODEL_URL -O shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2  # Extract the file


--2025-02-23 21:17:53--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2025-02-23 21:17:53--  https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  38.9MB/s    in 1.6s    

2025-02-23 21:17:55 (38.9 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [ ]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance as dist

# Load Dlib's face detector and the landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Define a function to calculate the Eye Aspect Ratio (EAR)
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])  # Vertical distance
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])  # Horizontal distance
    ear = (A + B) / (2.0 * C)
    return ear


In [ ]:
# Define eye landmarks (indexes from 68-point model)
(left_start, left_end) = (42, 48)
(right_start, right_end) = (36, 42)

EYE_AR_THRESH = 0.25  # EAR threshold for blink detection
EYE_AR_CONSEC_FRAMES = 3  # Number of consecutive frames for a blink

blink_counter = 0

cap = cv2.VideoCapture(0)  # Open webcam

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)
        landmarks = np.array([[p.x, p.y] for p in landmarks.parts()])

        left_eye = landmarks[left_start:left_end]
        right_eye = landmarks[right_start:right_end]

        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        ear = (left_ear + right_ear) / 2.0

        # Draw eyes
        cv2.polylines(frame, [left_eye], True, (0, 255, 0), 1)
        cv2.polylines(frame, [right_eye], True, (0, 255, 0), 1)

        # Check if EAR is below threshold
        if ear < EYE_AR_THRESH:
            blink_counter += 1
        else:
            if blink_counter >= EYE_AR_CONSEC_FRAMES:
                print("Blink detected!")
            blink_counter = 0

    cv2.imshow("Blink Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np

def capture_webcam():
    js_code = """
        async function captureWebcam() {
            const video = document.createElement('video');
            video.style.display = 'none';
            document.body.appendChild(video);

            const stream = await navigator.mediaDevices.getUserMedia({ video: true });
            video.srcObject = stream;
            await video.play();

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);

            stream.getTracks().forEach(track => track.stop());
            return canvas.toDataURL('image/jpeg', 0.8);
        }
        captureWebcam();
    """
    return eval_js(js_code)


In [ ]:
def process_webcam_frame():
    data_uri = capture_webcam()
    image_bytes = b64decode(data_uri.split(',')[1])

    # Convert to NumPy array
    nparr = np.frombuffer(image_bytes, np.uint8)
    frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect face
    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)
        landmarks = np.array([[p.x, p.y] for p in landmarks.parts()])

        left_eye = landmarks[left_start:left_end]
        right_eye = landmarks[right_start:right_end]

        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        ear = (left_ear + right_ear) / 2.0

        print(f"EAR: {ear:.2f}")
        if ear < EYE_AR_THRESH:
            print("Blink detected!")

# Run the function
process_webcam_frame()


EAR: 0.15
Blink detected!


In [ ]:
import time

# Parameters for drowsiness detection
DROWSINESS_THRESHOLD = 10  # Adjust based on testing
CHECK_INTERVAL = 60  # Check every 60 seconds

blink_counter = 0
start_time = time.time()

def process_webcam_frame():
    global blink_counter, start_time

    data_uri = capture_webcam()
    image_bytes = b64decode(data_uri.split(',')[1])

    # Convert to NumPy array
    nparr = np.frombuffer(image_bytes, np.uint8)
    frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect face
    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)
        landmarks = np.array([[p.x, p.y] for p in landmarks.parts()])

        left_eye = landmarks[left_start:left_end]
        right_eye = landmarks[right_start:right_end]

        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        ear = (left_ear + right_ear) / 2.0

        print(f"EAR: {ear:.2f}")

        # Check if EAR is below threshold (blink detected)
        if ear < EYE_AR_THRESH:
            blink_counter += 1
            print("Blink detected!")

    # Calculate elapsed time
    elapsed_time = time.time() - start_time

    # Check blink count every CHECK_INTERVAL seconds
    if elapsed_time > CHECK_INTERVAL:
        if blink_counter < DROWSINESS_THRESHOLD:
            print("⚠️ Drowsiness Detected! Please take a break.")

        # Reset counters
        blink_counter = 0
        start_time = time.time()

# Run the function every time to capture a frame and check blinks
process_webcam_frame()


EAR: 0.15
Blink detected!


In [ ]:
import time
import threading

# Parameters for drowsiness detection
DROWSINESS_THRESHOLD = 8  # Blinks per interval (adjust as needed)
CHECK_INTERVAL = 30  # Time window in seconds
FRAME_INTERVAL = 5  # Capture a frame every 5 seconds
RUN_DURATION = 60  # Stop after 60 seconds

blink_counter = 0
start_time = time.time()

def process_webcam_frame():
    global blink_counter, start_time

    data_uri = capture_webcam()
    image_bytes = b64decode(data_uri.split(',')[1])

    # Convert to NumPy array
    nparr = np.frombuffer(image_bytes, np.uint8)
    frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect face
    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)
        landmarks = np.array([[p.x, p.y] for p in landmarks.parts()])

        left_eye = landmarks[left_start:left_end]
        right_eye = landmarks[right_start:right_end]

        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        ear = (left_ear + right_ear) / 2.0

        print(f"EAR: {ear:.2f}")

        # Check if EAR is below threshold (blink detected)
        if ear < EYE_AR_THRESH:
            blink_counter += 1
            print("Blink detected!")

    # Calculate elapsed time
    elapsed_time = time.time() - start_time

    # Check blink count every CHECK_INTERVAL seconds
    if elapsed_time > CHECK_INTERVAL:
        print(f"🔹 Blinks in {CHECK_INTERVAL} sec: {blink_counter}")

        if blink_counter < DROWSINESS_THRESHOLD:
            print("⚠️ Drowsiness Detected! Please take a break.")

        # Reset counters
        blink_counter = 0
        start_time = time.time()

# Function to automate frame capturing every few seconds
def start_drowsiness_detection():
    start_time = time.time()
    while time.time() - start_time < RUN_DURATION:
        process_webcam_frame()
        time.sleep(FRAME_INTERVAL)  # Wait before capturing next frame

# Start automatic detection
start_drowsiness_detection()


EAR: 0.35
EAR: 0.25
EAR: 0.27
EAR: 0.20
Blink detected!
EAR: 0.26
EAR: 0.15
Blink detected!
🔹 Blinks in 30 sec: 2
⚠️ Drowsiness Detected! Please take a break.
EAR: 0.10
Blink detected!
EAR: 0.11
Blink detected!


In [ ]:
import time
import numpy as np
import cv2
import dlib
from scipy.spatial import distance
from google.colab.patches import cv2_imshow
from base64 import b64decode
from IPython.display import display, Javascript

# Load face detector and facial landmarks predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# EAR threshold for blink detection
EYE_AR_THRESH = 0.20

# MAR threshold for yawn detection
MOUTH_AR_THRESH = 0.7  # Increased threshold to reduce false yawns
YAWN_CONSEC_FRAMES = 3  # Mouth must stay open for 3 frames to count as yawn
yawn_frames = 0  # Counter for consecutive yawn frames

# Parameters for drowsiness detection
DROWSINESS_THRESHOLD = 8  # Blinks per interval
CHECK_INTERVAL = 30  # Seconds
FRAME_INTERVAL = 5  # Capture frame every 5 sec
RUN_DURATION = 60  # Total runtime

blink_counter = 0
yawn_counter = 0
start_time = time.time()

# Function to compute EAR (Eye Aspect Ratio)
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

# Function to compute MAR (Mouth Aspect Ratio)
def mouth_aspect_ratio(mouth):
    A = distance.euclidean(mouth[1], mouth[7])  # Vertical distance
    B = distance.euclidean(mouth[2], mouth[6])
    C = distance.euclidean(mouth[3], mouth[5])
    D = distance.euclidean(mouth[0], mouth[4])  # Horizontal distance
    return (A + B + C) / (3.0 * D)

# Function to process webcam frames
def process_webcam_frame():
    global blink_counter, yawn_counter, yawn_frames, start_time

    # Capture frame from webcam
    data_uri = capture_webcam()
    image_bytes = b64decode(data_uri.split(',')[1])
    nparr = np.frombuffer(image_bytes, np.uint8)
    frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect face
    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)
        landmarks = np.array([[p.x, p.y] for p in landmarks.parts()])

        # Extract eye landmarks
        left_eye = landmarks[36:42]
        right_eye = landmarks[42:48]

        # Extract mouth landmarks
        mouth = landmarks[48:68]

        # Calculate EAR (Eye Aspect Ratio)
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        ear = (left_ear + right_ear) / 2.0

        # Calculate MAR (Mouth Aspect Ratio)
        mar = mouth_aspect_ratio(mouth)

        print(f"EAR: {ear:.2f}, MAR: {mar:.2f}")

        # Detect Blink
        if ear < EYE_AR_THRESH:
            blink_counter += 1
            print("Blink detected!")

        # Detect Yawn only if the mouth stays open for consecutive frames
        if mar > MOUTH_AR_THRESH:
            yawn_frames += 1
        else:
            yawn_frames = 0  # Reset if mouth closes

        if yawn_frames >= YAWN_CONSEC_FRAMES:
            yawn_counter += 1
            print("😮 Yawn detected!")

        # Draw landmarks for debugging
        for (x, y) in mouth:
            cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)  # Green dots for mouth landmarks

    # Show frame with landmarks (for debugging)
    cv2_imshow(frame)

    # Check blink count every CHECK_INTERVAL seconds
    elapsed_time = time.time() - start_time
    if elapsed_time > CHECK_INTERVAL:
        print(f"🔹 Blinks in {CHECK_INTERVAL} sec: {blink_counter}")
        print(f"🔹 Yawns in {CHECK_INTERVAL} sec: {yawn_counter}")

        if blink_counter < DROWSINESS_THRESHOLD or yawn_counter >= YAWN_CONSEC_FRAMES:
            print("⚠️ Drowsiness Detected! Please take a break.")

        # Reset counters
        blink_counter = 0
        yawn_counter = 0
        start_time = time.time()

# Function to automate frame capturing
def start_drowsiness_detection():
    start_time = time.time()
    while time.time() - start_time < RUN_DURATION:
        process_webcam_frame()
        time.sleep(FRAME_INTERVAL)

# Start automatic detection
start_drowsiness_detection()


NameError: name 'capture_webcam' is not defined